In [2]:
from kubernetes import client, config, watch
namespace_name="zerok"

In [3]:
# Configs can be set in Configuration class directly or using helper utility
config.load_kube_config()

core_api = client.CoreV1Api()
apps_api = client.AppsV1Api()
watcher = watch.Watch()

In [ ]:
print(config.list_kube_config_contexts())

In [ ]:
namespace = client.V1Namespace(
    api_version = "v1",
    kind = "Namespace",
    metadata = client.V1ObjectMeta(name=namespace_name)
)
core_api.create_namespace(namespace)

In [ ]:
container = client.V1Container(
    name="nodeexample",
    image="301129966109.dkr.ecr.us-east-2.amazonaws.com/nodeexample",
    ports = [client.V1ContainerPort(container_port=80)],
)

In [ ]:
template=client.V1PodTemplateSpec(
    metadata = client.V1ObjectMeta(labels={"app": "target"}),
    spec = client.V1PodSpec(
        containers = [container]
    )
)

In [ ]:
spec = client.V1DeploymentSpec(
    selector=client.V1LabelSelector(match_labels={"app":"target"}),
    template=template,
    replicas = 1
)

In [ ]:
deployment = client.V1Deployment(
    api_version = "apps/v1",
    kind = "Deployment",
    metadata = client.V1ObjectMeta(name="target-nodeexample"),
    spec=spec
)

In [ ]:
apps_api.create_namespaced_deployment(namespace=namespace_name, body=deployment)

In [ ]:
deployment.spec.template.spec.containers[0].image = "301129966109.dkr.ecr.us-east-2.amazonaws.com/nodeexample"
limits = {'cpu':'145m', 'memory':'150Mi'}
requests = {'cpu':'125m', 'memory':'120Mi'}
deployment.spec.template.spec.containers[0].resources = client.V1ResourceRequirements(limits=limits, requests=requests)
apps_api.replace_namespaced_deployment(name="target-nodeexample", namespace=namespace_name, body=deployment)

In [4]:
core_api.delete_namespace(namespace_name)

{'api_version': 'v1',
 'code': None,
 'details': None,
 'kind': 'Namespace',
 'message': None,
 'metadata': {'_continue': None,
              'remaining_item_count': None,
              'resource_version': '5160',
              'self_link': None},
 'reason': None,
 'status': "{'phase': 'Terminating'}"}